In [ ]:
!pip install torchinfo
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import os
from torchvision import datasets, transforms
from torch.utils.data import random_split

import matplotlib.pyplot as plt
from torchinfo import summary

from sklearn.metrics import f1_score, confusion_matrix

from tqdm import tqdm
import pandas as pd
import sys
import torch.nn.functional as F


# Setting the working directory:

if not('Nilly' in os.getcwd() or '472data' in os.getcwd()):
  # os.chdir('drive/MyDrive/Nilly/')
  os.chdir('drive/MyDrive/Nilly/')

In [ ]:
EPOCHS = 60
lr = 0.001
b_s = 40

try_num = 3

In [ ]:
dir = 'COMP472-AK_15-main/'
save_dir = 'model_' + str(try_num) + '/'

if not os.path.isdir(save_dir):
  os.mkdir(save_dir)

In [ ]:
transform = transforms.Compose([
  transforms.Resize([224, 224]),
  # transforms.CenterCrop(224),
  transforms.Grayscale(num_output_channels=1),
  transforms.ToTensor(),
  # transforms.RandomRotation(30),
  # transforms.RandomHorizontalFlip(),
  transforms.Normalize(mean=[0.5], std=[0.2]), # Use single-channel values for grayscale
  ])
test_transform = transforms.Compose([
  transforms.Resize([224, 224]),
  transforms.Grayscale(num_output_channels=1),
  # transforms.CenterCrop(255),
  transforms.ToTensor(),
  transforms.Normalize(mean=[0.5], std=[0.2]), # Use single-channel values for grayscale
  ])

train_dataset = datasets.ImageFolder(root=dir + 'dataset/train', transform=transform)
test_dataset = datasets.ImageFolder(root=dir + 'dataset/test', transform=transform)

# train_data, val_data = train_test_split(train_dataset, test_size=0.2, random_state=42, stratify=train_dataset.targets)

In [ ]:
# prompt: split the train_dataset into validation set and train set

# Split the train_dataset into train and validation sets
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_data, val_data = random_split(train_dataset, [train_size, val_size])


In [ ]:
# prompt: make torch data loader from the dataset of the last block

train_dataloader = DataLoader(train_dataset, batch_size=b_s, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=b_s, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=b_s, shuffle=True)

In [ ]:
images, labels = next(iter(train_dataloader))

In [ ]:
class ConvNet7(nn.Module):
    def __init__(self, num_classes=5):
        super(ConvNet7, self).__init__()
        # Define the convolutional layer
        self.conv_net = nn.Sequential(

                                 nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(7, 7), stride=1, padding='same'),
                                 nn.BatchNorm2d(32),
                                 nn.ReLU(),
                                 nn.Dropout(0.3),

                                 nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(7, 7), stride=1, padding='same'),
                                 nn.BatchNorm2d(64),
                                 nn.ReLU(),

                                 nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2)),
                                 nn.Dropout(0.3),

                                 nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(7, 7), stride=1, padding='same'),
                                 nn.BatchNorm2d(128),
                                 nn.ReLU(),
                                 nn.Dropout(0.3),

                                 nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(7, 7), stride=1, padding='same'),
                                 nn.BatchNorm2d(128),
                                 nn.ReLU(),

                                 nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2)),
                                 nn.Dropout(0.3),

                                 # add layers:
                                 nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(7, 7), stride=1, padding='same'),
                                 nn.BatchNorm2d(128),
                                 nn.ReLU(),
                                 nn.Dropout(0.3),

                                 nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(7, 7), stride=1, padding='same'),
                                 nn.BatchNorm2d(128),
                                 nn.ReLU(),
                                 nn.Dropout(0.3),

                                 nn.Conv2d(in_channels=128, out_channels=64, kernel_size=(7, 7), stride=1, padding='same'),
                                 nn.BatchNorm2d(64),
                                 nn.ReLU(),
                                 nn.Dropout(0.3),

                                 nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2)),
                                 nn.Dropout(0.3),

                                 nn.Conv2d(in_channels=64, out_channels=32, kernel_size=(7, 7), stride=1, padding='same'),
                                 nn.BatchNorm2d(32),
                                 nn.ReLU(),
                                 nn.Dropout(0.3),

                                 nn.Conv2d(in_channels=32, out_channels=16, kernel_size=(7, 7), stride=1, padding='same'),
                                 nn.BatchNorm2d(16),
                                 nn.ReLU(),

                                 nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2)),
                                 nn.Dropout(0.3),

                                 nn.Conv2d(in_channels=16, out_channels=8, kernel_size=(7, 7), stride=1, padding='same'),
                                 nn.BatchNorm2d(8),
                                 nn.ReLU(),

                                 nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2)),
                                 nn.Dropout(0.3),

                                 )
        self.fc = nn.Sequential(
                                nn.Linear(8 * 7 * 7, 128),
                                nn.ReLU(),

                                nn.Linear(128, 64),
                                nn.ReLU(),

                                nn.Linear(64, num_classes),
                                nn.ReLU(),
                                )




    def forward(self, x):
        # Pass the input through the convolutional layer
        x = self.conv_net(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        x = F.softmax(x, dim=1)
        return x

# model = ConvNet()
# output = model(images)


In [ ]:
model = ConvNet7()
# output = model(images)
summary(model, input_size=(b_s, 1, 224, 224))

Layer (type:depth-idx)                   Output Shape              Param #
ConvNet7                                 [40, 5]                   --
├─Sequential: 1-1                        [40, 8, 7, 7]             --
│    └─Conv2d: 2-1                       [40, 32, 224, 224]        1,600
│    └─BatchNorm2d: 2-2                  [40, 32, 224, 224]        64
│    └─ReLU: 2-3                         [40, 32, 224, 224]        --
│    └─Dropout: 2-4                      [40, 32, 224, 224]        --
│    └─Conv2d: 2-5                       [40, 64, 224, 224]        100,416
│    └─BatchNorm2d: 2-6                  [40, 64, 224, 224]        128
│    └─ReLU: 2-7                         [40, 64, 224, 224]        --
│    └─MaxPool2d: 2-8                    [40, 64, 112, 112]        --
│    └─Dropout: 2-9                      [40, 64, 112, 112]        --
│    └─Conv2d: 2-10                      [40, 128, 112, 112]       401,536
│    └─BatchNorm2d: 2-11                 [40, 128, 112, 112]       256


In [ ]:
class ED_model(nn.Module):
    def __init__(self, in_channels=1, out_channels=5):
        super(ED_model, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding='same', bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.dropout1 = nn.Dropout(p=0.3)

        self.conv2 = nn.Conv2d(in_channels=16, out_channels=64, kernel_size=3, stride=1, padding='same', bias=False)
        self.bn2 = nn.BatchNorm2d(64)
        self.dropout2 = nn.Dropout(p=0.3)

        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding='same', bias=False)
        self.bn3 = nn.BatchNorm2d(128)
        self.max_pool1 = nn.MaxPool2d(kernel_size=2, stride=2) # 48x48 -> 24x24
        self.dropout3 = nn.Dropout(p=0.3)

        self.conv4 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding='same', bias=False)
        self.bn4 = nn.BatchNorm2d(128)
        self.dropout4 = nn.Dropout(p=0.3)

        self.conv5 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding='same', bias=False)
        self.bn5 = nn.BatchNorm2d(128)
        self.dropout5 = nn.Dropout(p=0.3)

        self.conv6 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding='same', bias=False)
        self.bn6 = nn.BatchNorm2d(128)
        self.max_pool2 = nn.MaxPool2d(kernel_size=2, stride=2) # 24x24 -> 12x12
        self.dropout6 = nn.Dropout(p=0.3)

        self.conv7 = nn.Conv2d(in_channels=128, out_channels=64, kernel_size=3, stride=2, padding=1, bias=False)
        self.bn7 = nn.BatchNorm2d(64)
        self.dropout7 = nn.Dropout(p=0.3)

        self.conv8 = nn.Conv2d(in_channels=64, out_channels=16, kernel_size=3, stride=2, padding=0, bias=False)
        self.bn8 = nn.BatchNorm2d(16)
        self.max_pool3 = nn.MaxPool2d(kernel_size=2, stride=2) # 12x12 -> 6x6
        self.dropout8 = nn.Dropout(p=0.3)

        self.fc1 = nn.Linear(in_features=6*6*16, out_features=256)
        self.fc2 = nn.Linear(in_features=256, out_features=32)
        self.fc3 = nn.Linear(32, out_channels)


    def forward(self, x):

        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.dropout1(x) # <- block 1
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.dropout2(x) # <- block 2
        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)
        x = self.max_pool1(x)
        x = self.dropout3(x) # <- block 3

        x = self.conv4(x)
        x = self.bn4(x)
        x = F.relu(x)
        x = self.dropout4(x) # <- block 4
        x = self.conv5(x)
        x = self.bn5(x)
        x = F.relu(x)
        x = self.dropout5(x) # <- block 5
        x = self.conv6(x)
        x = self.bn6(x)
        x = F.relu(x)
        x = self.max_pool2(x)
        x = self.dropout6(x) # <- block 6

        x = self.conv7(x)
        x = self.bn7(x)
        x = F.relu(x)
        x = self.dropout7(x) # <- block 7
        x = self.conv8(x)
        x = self.bn8(x)
        x = F.relu(x)
        x = self.max_pool3(x)
        x = self.dropout8(x) # <- block 8

        x = torch.flatten(x, start_dim=1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = F.softmax(x, dim=1)

        return x

In [ ]:
model = ED_model(in_channels=1, out_channels=5)
summary(model, input_size=(32, 1, 224, 224))

Layer (type:depth-idx)                   Output Shape              Param #
ED_model                                 [32, 5]                   --
├─Conv2d: 1-1                            [32, 16, 224, 224]        144
├─BatchNorm2d: 1-2                       [32, 16, 224, 224]        32
├─Dropout: 1-3                           [32, 16, 224, 224]        --
├─Conv2d: 1-4                            [32, 64, 224, 224]        9,216
├─BatchNorm2d: 1-5                       [32, 64, 224, 224]        128
├─Dropout: 1-6                           [32, 64, 224, 224]        --
├─Conv2d: 1-7                            [32, 128, 224, 224]       73,728
├─BatchNorm2d: 1-8                       [32, 128, 224, 224]       256
├─MaxPool2d: 1-9                         [32, 128, 112, 112]       --
├─Dropout: 1-10                          [32, 128, 112, 112]       --
├─Conv2d: 1-11                           [32, 128, 112, 112]       147,456
├─BatchNorm2d: 1-12                      [32, 128, 112, 112]       256

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)

cuda:0


In [ ]:
total_train_losses = []
total_val_losses = []
total_train_accuracy = []
total_val_accuracy = []
total_train_f1 = []
total_val_f1 = []
learning_rate_tracker = []

def train(model, optimizer, criterion, scheduler, EPOCHS):

  cur_loss = 9999

  for epoch in tqdm(range(1, EPOCHS+1)):

    # Train model
    model.train()
    train_losses = []
    train_accuracy = []
    train_f1 = []

    for i, batch in enumerate(train_dataloader):
      optimizer.zero_grad()

      img_batch, label_batch = batch   #img [B,3,H,W], label[B,N_CLASSES]
      img_batch = img_batch.to(device)
      label_batch = label_batch.type(torch.LongTensor).to(device)

      #Train model
      output = model(img_batch) # output: [B, 7, H, W]


      # print(img_batch)
      loss = criterion(output, label_batch)
      loss.backward()
      optimizer.step()

      #Add current loss to temporary list (after 1 epoch take avg of all batch losses)
      preds = torch.argmax(torch.exp(output), dim=1)

      f1 = f1_score(preds.cpu(), label_batch.cpu(), average='macro')
      acc = torch.sum(preds == label_batch) / len(preds)
      train_losses.append(loss.item())
      train_accuracy.append(acc.cpu())
      train_f1.append(f1)
      print('\r', f'TRAIN Epoch: {epoch}, batch: {i} | Batch metrics | loss: {loss.item():.4f}, f1: {f1:.3f}, accuracy: {acc:.3f}', end='', flush=True)


    print()
    print(f'TRAIN Epoch: {epoch} | Epoch metrics | loss: {np.mean(train_losses):.4f}, f1: {np.mean(train_f1):.3f}, accuracy: {np.mean(train_accuracy):.3f}, learning rate: {optimizer.state_dict()["param_groups"][0]["lr"]:.6f}')
    total_train_losses.append(np.mean(train_losses))
    total_train_accuracy.append(np.mean(train_accuracy))
    total_train_f1.append(np.mean(train_f1))

    #Update learning rate
    learning_rate_tracker.append(optimizer.state_dict()['param_groups'][0]['lr'])
    scheduler.step()

    # Validate model
    model.eval()
    val_losses = []
    val_accuracy = []
    val_f1 = []

    for i, batch in enumerate(val_dataloader):
      #Extract data, labels
      img_batch, label_batch = batch
      img_batch = img_batch.to(device)
      label_batch = label_batch.type(torch.LongTensor).to(device)

      #Validate model
      with torch.cuda.amp.autocast():
          output = model(img_batch)
          loss = criterion(output, label_batch)

      #Add current loss to temporary list (after 1 epoch take avg of all batch losses)
      preds = torch.argmax(output, dim=1)
      f1 = f1_score(preds.cpu(), label_batch.cpu(), average='macro')
      acc = torch.sum(preds == label_batch) / len(preds)
      val_losses.append(loss.item())
      val_accuracy.append(acc.cpu())
      val_f1.append(f1)


    # Update global metrics
    print(f'VALIDATION  Epoch: {epoch} | Epoch metrics | loss: {np.mean(val_losses):.4f}, f1: {np.mean(val_f1):.3f}, accuracy: {np.mean(val_accuracy):.3f}')
    print('-'*50)
    total_val_losses.append(np.mean(val_losses))
    total_val_accuracy.append(np.mean(val_accuracy))
    total_val_f1.append(np.mean(val_f1))

    # Save the results so far
    temp_df = pd.DataFrame(list(zip(total_train_losses, total_val_losses, total_train_f1, total_val_f1,
                                total_train_accuracy, total_val_accuracy)),
                        columns = ['train_loss', 'val_loss', 'train_f1', 'test_f1', 'train_accuracy',
                                  'test_accuracy'])
    temp_df.to_csv(save_dir + 'train_val_measures')

    if np.mean(val_losses) < cur_loss:
      print('saving model')
      torch.save(model.state_dict(), save_dir + 'model.pt')
      cur_loss = np.mean(val_losses)


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1) # Use dynamic learning rate
criterion = nn.CrossEntropyLoss().to(device)

In [ ]:
train(model, optimizer, criterion, scheduler, EPOCHS)

  0%|          | 0/60 [00:00<?, ?it/s]

 TRAIN Epoch: 1, batch: 70 | Batch metrics | loss: 1.6094, f1: 0.000, accuracy: 0.000
TRAIN Epoch: 1 | Epoch metrics | loss: 1.6095, f1: 0.061, accuracy: 0.163, learning rate: 0.001000


  2%|▏         | 1/60 [20:09<19:49:48, 1209.98s/it]

VALIDATION  Epoch: 1 | Epoch metrics | loss: 1.6094, f1: 0.050, accuracy: 0.148
--------------------------------------------------
saving model
 TRAIN Epoch: 2, batch: 70 | Batch metrics | loss: 1.6094, f1: 0.042, accuracy: 0.091
TRAIN Epoch: 2 | Epoch metrics | loss: 1.6094, f1: 0.056, accuracy: 0.167, learning rate: 0.001000


  3%|▎         | 2/60 [21:30<8:47:31, 545.72s/it]  

...

VALIDATION  Epoch: 58 | Epoch metrics | loss: 1.6094, f1: 0.067, accuracy: 0.169
--------------------------------------------------
 TRAIN Epoch: 59, batch: 70 | Batch metrics | loss: 1.6094, f1: 0.086, accuracy: 0.273
TRAIN Epoch: 59 | Epoch metrics | loss: 1.6094, f1: 0.057, accuracy: 0.169, learning rate: 0.000010


 98%|█████████▊| 59/60 [1:30:50<01:12, 72.36s/it]

VALIDATION  Epoch: 59 | Epoch metrics | loss: 1.6094, f1: 0.061, accuracy: 0.169
--------------------------------------------------
 TRAIN Epoch: 60, batch: 70 | Batch metrics | loss: 1.6094, f1: 0.086, accuracy: 0.273
TRAIN Epoch: 60 | Epoch metrics | loss: 1.6094, f1: 0.057, accuracy: 0.169, learning rate: 0.000010


100%|██████████| 60/60 [1:32:02<00:00, 92.04s/it]

VALIDATION  Epoch: 60 | Epoch metrics | loss: 1.6094, f1: 0.050, accuracy: 0.148
--------------------------------------------------
